### Instructions
A - above
B - below
X - delete 
Ctrl Enter - run
Enter - Enter into cell
M - make cell into markdown cell

In [6]:
import numpy as np
import pandas as pd
from matplotlib import pyplot

In [117]:
f = open("modules.txt", 'r')
#print(f.read())
f.close()

# THIS IS A HEADING. ITS NEAT.
# BELOW IT IS SOME CODE.

In [214]:
def generate_timetable():
    """
    Generate a timetable Spyx
    Read the file to see how many classes + labs there are.
    Store them in a 20x3 list alongside empty sessions to fill the list
    """
    #Create "empty" timetable
    timetable = [["empty" for x in range(3)] for y in range(20)] 
    
    #Read file into allSplits list
    allSplits = []
    f = open("modules.txt", 'r')
    lineCount = 0
    for line in f:
        allSplits.append(line.split("|"))
        lineCount+=1
    f.close()
    
    #Add module into timetable
    for i in range(0,lineCount):
        timetable[i][0] = allSplits[i][0]
    
    
    #Add seminars into timetable.
    #Iterate through each module, for each number of seminars use
    #counters to iterate to next empty position and place seminar.
    #Should work on different set of modules if format is kept identical.
    emptyChecker = 0
    emptyChecker1 = 0
    for i in range(0, lineCount):
        for j in range(0, int(allSplits[i][2])):
            toggle = False
            while toggle == False:
                if timetable[emptyChecker][emptyChecker1] == "empty":
                    timetable[emptyChecker][emptyChecker1] = allSplits[i][0]
                    toggle = True
                emptyChecker1+=1
                if emptyChecker1 == 3:
                    emptyChecker+=1
                    emptyChecker1=0
    
    #randomises through randomise function below
    randomSwitch = False
    while randomSwitch == False:
        randomise_timetable(timetable)
        randomSwitch = room_validity(timetable)

    #for i in range(0, len(timetable)):
        #print(timetable[i])
    #print(timetableJoined)
    return timetable

generate_timetable()

[['MOD005', 'MOD001', 'MOD008'],
 ['MOD010', 'MOD002', 'MOD006'],
 ['MOD002', 'MOD004', 'MOD006'],
 ['MOD016', 'MOD012', 'MOD012'],
 ['MOD017', 'MOD015', 'MOD005'],
 ['MOD009', 'empty', 'MOD008'],
 ['empty', 'MOD011', 'MOD005'],
 ['MOD001', 'MOD011', 'MOD009'],
 ['MOD015', 'empty', 'MOD015'],
 ['MOD012', 'MOD009', 'MOD010'],
 ['MOD006', 'MOD016', 'MOD010'],
 ['MOD014', 'MOD014', 'MOD001'],
 ['MOD011', 'MOD013', 'empty'],
 ['MOD013', 'empty', 'MOD014'],
 ['empty', 'empty', 'MOD009'],
 ['MOD007', 'MOD017', 'MOD006'],
 ['MOD004', 'MOD002', 'MOD007'],
 ['empty', 'empty', 'empty'],
 ['MOD003', 'MOD003', 'empty'],
 ['MOD008', 'MOD003', 'empty']]

In [208]:
def randomise_timetable(timetableI):
    #Create randomised timetable
    #Add all to single list
    #Shuffle list
    #Put back into timetable
    
    timetableJoined = []
    counter = 0
    for list in timetableI:
        for i in list:
            timetableJoined.append(i)
    np.random.shuffle(timetableJoined)
    for i in range(0,len(timetableI)):
        for j in range(0,3):
            timetableI[i][j] = timetableJoined[counter]
            counter+=1
    
    return timetableI

In [215]:
def room_validity(timetable):
    #Check to see if timetable is valid ie:
    #lectures take place in lecture theatre & labs are in labs
    #returns true if valid, else false if not
    allSplits = []
    f = open("modules.txt", 'r')
    for line in f:
        allSplits.append(line.split("|"))
    f.close()
    counter = -1
    for i in range(0,len(timetable)):
        current = timetable[i][0]
        if current=="empty":
            continue
        for j in range(0,len(timetable)):
            if i!=j:
                if current == timetable[j][0]:
                    if counter < 0:
                        counter = j
                    else:
                        return False
    
    #Checks that all lectures are in lecutre theatre (and therefore labs are in labs)
    allClasses = 0;
    for l in range(0,len(allSplits)):
        tree = allSplits[l][0]
        for m in range(0,len(timetable)):
            if allSplits[l][0] == timetable[m][0]:
                allClasses+=1
                break
    if allClasses != 17:
        return False
    return True             

In [ ]:
def fitness_function():
    """
    Generate a fitness calculator Spyx
    General setup directly below.
    Could probably be more efficient if
    unnecessary repetition was removed.
    """
    allSplits = []
    f = open("modules.txt", 'r')
    for line in f:
        allSplits.append(line.split("|"))
    f.close()
    
    timetableV = generate_timetable()
    for i in range(0, len(timetableV)):
        print(timetableV[i])
    
    """
    CONSTRAINTS:
    1) ONLY 1 LECTURE AND TWO LABS MAY RUN CONCURRENTLY
    2) LABS FOR A MODULE MAY ONLY OCCUR AFTER ITS LECTURE! PRECEDENCE!
    3) SESSIONS CAN'T INTERFERE! CONCURRENCE!
    """
    #Precedence constraints
    precConstraints = 0
    #module loop
    for i in range(0, len(timetableV)):
        module = timetableV[i][0]
        #all of the timetable lab loop
        for j in range(0, len(timetableV)):
            if j > i:
                break
            for l in range(1, len(timetableV[j])):
                if module == timetableV[j][l]:
                    precConstraints+=1
            
    
    #Concurrency constraints
    concConstraints = 0
    #for i in range(0, len(timetableV)):
        
    print(precConstraints)
fitness_function()
    